In [ ]:
# Notebook for the analysis of vibration measurements.
#
# TU Delft QuTech/HanslonLab, Wouter Westerveld, October 2017
# 
# oct2017 version is the last version by Wouter. Older versions (sep2017, sep2017_new) do include some analysis 
# that are not in this version).
#
# Measurement setup: 
#   laser --> cavity fiber --> cavity --> free-space objective --> photo-diode --> yokogawa ossiloscope
#
# Ossiloscope files:
#   Store as .wdf file. 
#   Copy to data folder on PC (possibly using this notebook)
#   Convert .wdf files to .h5 files using Matlab script 
#     D:\measuring\measurement\scripts\cav1_scripts\tools\ConvertWdfToHDF5.m
#   NOTE that the .h5 file dataset CH0, CH1, CH2, etc are simply the first stored channels. 
#     These do not correspond tho the channels of the ossiloscope. E.g. if the ossiloscope channels CH1, CH2, Ch5 are
#     stored then they are called CH0, CH1, CH2, respectively in the h5 file. 
#     The h5 dataset has an attribute TraceName which indicate which ossiloscope trace is stored.
#   NOTE that the .h5 dataset are 16-bit integers as internally used in the ossiloscope. 
#     Data in V is data * VResolution + VOffset (attributes of h5 dataset)
#     
# Notebook chapters:
#   Functions to load xviewer csv (from binary scope data) and scope csv (obsolete)
#   Functions to analize vibration time-traces
#   Update folder from scope
#     Simple cell for usage at CAV1 measurement PC, copies non-exising files from scope mounted as USB device to local data folder.
#   Measurements September and October 2017
#     Scripts to analyze vibration data.
#     Plot basic properties of files (data histogram, min, max values, etc.)
#       Typicaly you run this first to select traces to be analyzed further.
#     Estimate noise data
#       This cell only does something if noiseFilePathNames is not empty. 
#       Simulate how the photo-diode noise would translate to voltage noise at correct position of the flank. 
#       Simply add mean value of vibrationFilePathNames to noiseFilePathNames
#        Output is a new .h5 file that can be considered as a measured .h5 file for futher processing.
#     Convert voltage to dl
#     Compute power spectral density
#     Analyze drift (files 18 September, filter not ok for other files )    
#   Plot PSDs of different files
#     After running the analysis above (which you can only run for one dataset because variables such as cavity length are shared),
#     you can plot PSDs of different files wich are now stored in _analysis.h5 -> power 
#   Analysis 200 s measurements to find how much measurement time is avaliable with RMS < 0.1 nm and how to predict these times
#     You first hatve to run the script 'Measurements September and October 2017' to get the _analysis.h5 file with dataset 'dl'
#     Analysis of Montana cycles (also see PPTs). 
#     
#
# Notes: MemoryError
#   There is a memory leak in the voltage_to_dl cell. Processing multiple files show a increase of memory. Workaround is to 
#   process a limited number of files after each other. Then Kernel -> Restart. Then continue processing.
#   Note that this does not nessecarily show up when running this file. Especially computing spectra takes significant
#   memory and will give MemoryError if the memory was quite full already.
#   Also make sure you use File -> Close and Halt when exciting notebook. Otherwise python keeps running and
#   consuming memory.
#


import numpy as np
import os.path
import pandas
import matplotlib
import scipy.signal
import scipy.constants
from scipy.signal import butter, filtfilt
import sys
import h5py

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec


## Functions to load xviewer csv (from binary scope data) and scope csv (obsolete)

In [ ]:
def read_scope_xviewer_csv(filename):
    """ Read csv files exported using Yokogawa XViewer software
        
        Update 11-09-2017: auto-detect number of columns.        
    """
    if not( os.path.isfile(filename) ):
        print 'file does not exist.'
        return
    headerRows = 10        
    with open(filename) as f:                      
        for x in range(headerRows):
            line = f.readline()                        
            if "HResolution" in line:
                i1 = line.find(',')+1
                i2 = line.find(',',i1)                
                timeResolution = float( line[i1:i2].strip(' \t\n\r') )        
                numberOfCols = line.count(',')    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows,usecols=range(1,numberOfCols)).values
    return data, timeResolution


def read_scope_csv(filename):
    """ Read csv files as stored by Yokogawa Ossiloscope.                     
    """    
    headerRows = 16        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.rfind(',')+1:-1].strip(' \t\n\r') )    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows).values[:,1:-1]
    return data, timeResolution



## Functions to analize vibration time-traces

In [ ]:
from scipy.optimize import minimize
from scipy import interpolate

def lorentzian(x,p):    
    """
    Lorenzian function p[0] is center, p[1] is fwhm, p[2] is maximum intensity
    Similar to http://mathworld.wolfram.com/LorentzianFunction.html, but with p[2] maximum intensity, NOT integrated intensity of one.
    """
    return p[2] * (0.5 * p[1])**2 / ( ( x - p[0] )**2 + ( 0.5 * p[1] )**2 )

def lorentz_find_x(lorentzFwhm, lorentzMax, lorentzValue):    
    """ For lorenzian function y=f(x) with lorentzFwhm and lorentzMax centered at x=0, find x for y=lorentzValue. """
    def fun(x):
        return ( lorentzian( x, [0, lorentzFwhm, lorentzMax] ) -  lorentzValue ) ** 2    
    res = minimize( fun, lorentzFwhm,  method='Nelder-Mead' ) #  bounds=( (0, None ), ) does not work for Nelder-Mead
    if not( res.success ):
        print 'lorentzFindX convergence error: ', res.message       
    return np.absolute( res.x[0] )
    
def lorentz_find_flank_steepness(lorentzFwhm, lorentzMax, x):    
    """ Return steepness of flank for of lorentz dy/dx for given x. Assume lorentz centered at x=0. """  
    return lorentzMax * 8.0 * lorentzFwhm**2 * x / ( 4.0 * x**2 + lorentzFwhm**2 )**2
    

def linear_voltage_to_detuning(d, t, lorentzFwhm, lorentzMaxV):
    flankSteepness = lorentz_find_flank_steepness( lorentzFwhm, lorentzMaxV, lorentz_find_x( lorentzFwhm, lorentzMaxV, meanVInD ) )
    dHz = d / flankSteepness    
    return dHz

def rms_deviation( x ):
    """ return root-mean-square value of signal, with mean substracted """
    return np.sqrt( np.mean( (x - np.mean(x))**2 ) )
    
def lorentz_voltage_to_detuning(d, timeResolution, lorentzFwhm, lorentzMaxV, doPlot=False ):    
    meanVInD = np.mean( d ) 
    minVInD = np.amin(d)    
    # Compute vibrations in Hz by first applying lorentzian function as lookup table V -> dHz
    minDFreqInD = lorentz_find_x( lorentzFwhm, lorentzMaxV, minVInD )
    lorentzDFreq = np.linspace( -minDFreqInD-1e9, 0, 10000 )        
    lorentzV = lorentzian( lorentzDFreq, [0, lorentzFwhm, lorentzMaxV] )    
    # print minDFreqInD, np.amin( lorentzV ),  np.amax( lorentzV )
    dHz = interpolate.interp1d(lorentzV, lorentzDFreq)(d)  # will cause error if signal > lorentzMaxV  or if something went wrong with minDFreqInD
    # Print and plot results
    if doPlot:   
        i1 = 0
        i2 = min( 50000, len(d) )
        t = timeResolution * np.arange( i1, i2 )
        plt.figure(figsize=(14, 6))
        plt.subplot(131)
        plt.plot( t, d[i1:i2] )
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'transmission (V)' )
        plt.grid()
        plt.subplot(132)
        plt.plot( t, dHz[i1:i2]*1e-9 )
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'cavity detuning (GHz)' )
        plt.grid()
        plt.subplot(133)
        plt.plot( lorentzDFreq*1e-9, lorentzV )
        plt.xlabel( 'cavity detuning GHz' )
        plt.ylabel( 'V' )
        plt.grid()
        plt.show()
        plt.close()
    return dHz

def detrend_conv( dl, smoothWindowTime = 5.0, downsampleN = 50):

    def smooth(x,window_len=11,window='hanning'):
        """smooth the data using a window with requested size.

        This method is based on the convolution of a scaled window with the signal.
        The signal is prepared by introducing reflected copies of the signal 
        (with the window size) in both ends so that transient parts are minimized
        in the begining and end part of the output signal.

        input:
            x: the input signal 
            window_len: the dimension of the smoothing window; should be an odd integer
            window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
                flat window will produce a moving average smoothing.

        output:
            the smoothed signal

        example:

        t=linspace(-2,2,0.1)
        x=sin(t)+randn(len(t))*0.1
        y=smooth(x)

        see also: 

        numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
        scipy.signal.lfilter

        TODO: the window parameter could be the window itself if an array instead of a string
        NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
        """

        if x.ndim != 1:
            raise ValueError, "smooth only accepts 1 dimension arrays."
        if x.size < window_len:
            raise ValueError, "Input vector needs to be bigger than window size."
        if window_len<3:
            return x
        if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
            raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"

        s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]    
        if window == 'flat': #moving average
            w=numpy.ones(window_len,'d')
        else:
            w=eval('np.'+window+'(window_len)')

        y=np.convolve(w/w.sum(),s,mode='valid')    
        return y[(window_len/2-1):-(window_len/2)]
    
    
    if len( dl ) % downsampleN != 0:
        raise ValueError( 'len(dl) should be equal number times downsampleN' )
        
    smoothWindowN = int( smoothWindowTime / ( downsampleN * s['timeResolution'] ) )
    if ( smoothWindowN % 2 ) != 0: 
        smoothWindowN = smoothWindowN + 1    
    dlSmooth = smooth( dl.reshape(-1, downsampleN).mean(axis=1), smoothWindowN )
    dlDetrended = dl - np.interp(np.arange( len( dl ) ), np.arange( 0, len( dl ), downsampleN ), dlSmooth )        
    
    if False:
        t = np.arange( len( dlSmooth) ) * downsampleN * s['timeResolution']
        plt.plot( t, dl[::downsampleN]*1e9, label='time-trace' )         
        plt.plot( t, dlSmooth*1e9, label='smooth time-trace' ) 
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'dl (nm)' )
        plt.legend()
        plt.show()
        plt.close()    

    return dlDetrended
            
def calc_cum_pow( freq_pow, power, min_freq=0.1, max_freq=20e3 ):
    """ Compute spectra and cumtrapz 
    
        Input:
          freq_pow - frequency axis, 1D array
          power    - power, 1D array (as function of frequency)        
          min_freq - minimum frequency in analysis. DC power is power in frequencys below min_freq. Hz.
          max_freq - maximum frequency 
        
        Output:
          cum_freqs - frequency axis for cum_pow
          cum_pow   - cummulative/integrated power spectrum, using trapz
          full_pow  - power integrated over all frequencies
          dc_pow    - power in frequencies below min_freq

    """
    
    # min_freq = 0.1  # we do not take frequencies below 0.1 Hz into account.
    # max_freq = 20e3

    min_ind = np.argmin(np.abs(freq_pow-min_freq))
    max_ind = np.argmin(np.abs(freq_pow-max_freq))

    full_pow = scipy.integrate.trapz(power,freq_pow,axis=0)
    dc_pow = scipy.integrate.trapz(power[0:min_ind],freq_pow[0:min_ind],axis=0)

    cum_freqs = freq_pow[min_ind+1:max_ind]      
    cum_pow = scipy.integrate.cumtrapz( power[min_ind:max_ind], freq_pow[min_ind:max_ind] )    
    
    return cum_freqs, cum_pow, full_pow, dc_pow    

def dataset_in_h5file( fn, datasetName ):
    with h5py.File( fn ) as f:
        datasetInH5File = datasetName in f
    return datasetInH5File   


def read_h5file_attrs_as_dict( fn, datasetName = '/' ):
    s = {}
    with h5py.File( fn ) as f:    
        for key, value in f[datasetName].attrs.iteritems():
            s[key] = value    
    return s

def unique_list( seq ):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def ax_add_minortick( ax, Nx=5, Ny=5):                        
    ax.set_xticks( np.arange( ax.get_xticks()[0], ax.get_xticks()[-1], ( ax.get_xticks()[1] - ax.get_xticks()[0] ) / Nx ), minor=True)                                           
    ax.set_yticks( np.arange( ax.get_yticks()[0], ax.get_yticks()[-1], ( ax.get_yticks()[1] - ax.get_yticks()[0] ) / Ny ), minor=True)                                           
    ax.grid(which='minor', alpha=0.2)  

def get_time_axis( fn, timeStart=0, timeEnd=-1 ):
    """ Load / compute time axis 
    
        Input:
          fn            - filename including path
          timeStart=0   - starting time of axis, s
          timeEnd=-1    - end time of axis, s. timeEnd=-1 selects trace to end.          
        
        Output:
          time          - time axis
          i1            - only if timeStart or timeEnd are used, index of timeStart. 
          i2            - only if timeStart or timeEnd are used, index of timeEnd.           

    """    
    raise NameError( 'function needs update, still old data format' )
    s = pickle.load( open( fn + '_settings.p', 'rb' ) )            
    if timeEnd == -1:
        timeEnd = s['timeResolution'] * s['len']
    if timeStart < 0:
        raise NameError( 'timeStart earlier than 0' )        
    if timeEnd > s['timeResolution'] * s['len']:
        raise NameError('timeEnd later than trace length (use -1 to set it to trace length)' )        
    if timeStart >=  timeEnd:
        raise NameError(' timeEnd should be later than timeStart' )        
    i1 = int( timeStart / s['timeResolution'] )
    i2 = int( timeEnd / s['timeResolution'] )   
    time = s['timeResolution'] * np.arange( i1, i2 ) 
    if (timeStart == 0) and (timeEnd == -1):        
        return time
    else:
        return time, i1, i2

def plot_timetrace( fn, timeStart, timeEnd ):
    raise NameError( 'function needs update, still old data format' )
    data = np.load( fn + '_data.npy' )
    fileName = os.path.basename( fn )
    time, i1, i2 = get_time_axis( fn, timeStart, timeEnd )    
    plt.figure(figsize=(16, 8))
    plt.plot( time, data[i1:i2] )
    plt.grid()
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'V' )
    plt.title( fileName )
    plt.show()  
    plt.close()
    del data
    del time
    
def plot_histograms( fns, dataCH = 'CH0' ):
    for fn in unique_list( fns ):    
        if not os.path.isfile( fn ):
            raise NameError( 'file does not exist: ' + fn )        
        with h5py.File( fn ) as f:    
            data = f[dataCH] * f[dataCH].attrs['VResolution'] + f[dataCH].attrs['VOffset']                        
        fileName = os.path.basename( fn )
        plt.hist(data,20,label=fileName)
        print '{}, min {:6.3f} V, max {:6.3f} V, mean {:6.3f} V'.format( fileName, np.amin( data ), np.amax( data ), np.mean( data ) )  
    plt.legend()
    plt.grid()
    plt.show()
    plt.close()
    del data   
    
def convert_csv_file( fn, photodiodeScopeChannel=0, skipIfExists=True, timeStart=0, timeEnd=-1 ):
    # Store data as numpy array and dataSettings as pickle.    
    if skipIfExists and os.path.isfile( fn + '_data.npy' ):
        return
    # load data
    fileName = os.path.basename( fn )
    data, timeResolution = read_scope_xviewer_csv( fn )
    data = data[:,0]        
    # apply winow
    if ( (timeStart != 0) or (timeEnd != -1) ):        
        if timeEnd == -1:
            timeEnd = timeResolution * len( data )         
        if timeStart < 0:
            raise NameError( 'timeStart earlier than 0' )        
        if timeEnd > timeResolution * len( data ):
            raise NameError('timeEnd later than trace length (use -1 to set it to trace length)' )        
        if timeStart >=  timeEnd:
            raise NameError(' timeEnd should be later than timeStart' )        
        i1 = int( timeStart / timeResolution )
        i2 = int( timeEnd / timeResolution )     
        data = data[i1:i2]
        isWindowed = True
    else:
        isWindowed = False
    
    dataSettings = { 'fileName': fileName, 
                     'timeResolution': timeResolution,
                     'len': len( data ),
                     'min': np.amin( data ),
                     'max': np.amax( data ),
                     'mean': np.mean( data ),
                     'isWindowed': isWindowed,
                     'timeStart': timeStart,
                     'timeEnd': timeEnd }
    print '{} time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        fileName, timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), len(data) )   
    np.save( fn + '_data.npy', data )    
    pickle.dump( dataSettings, open( fn + '_settings.p', 'wb' ) )            
    del data, dataSettings    

    
def simulate_noise_data( vibrationFilePathNames, noiseFilePathNames, dataCH = 'CH0' ):
    ''' Simulate noise data, store in separate file. 
        
        Input:
            vibrationFilePathNames - array with vibration file names
            noiseFilePathNames     - corresponding array with noise filenames. Should have same length as vibrationFilePathNames.
            
        Output:            
            noiseSimulationFilePathNames - path names of generated files.
    '''
    noiseSimulationFilePathNames = []    
    if len( noiseFilePathNames ) > 0:
        for i in range( len( vibrationFilePathNames ) ):        
            fn = vibrationFilePathNames[i]        
            fnNoise = noiseFilePathNames[i]
            fnOut = fn + '_noise_simulation.h5'                                       
            with h5py.File( fn ) as f:    
                dataMean = np.sum( f[dataCH] ) / f[dataCH].shape[0] * f[dataCH].attrs['VResolution'] + f[dataCH].attrs['VOffset']                          
            with h5py.File( fnNoise ) as f:    
                dataSimulatedNoise = f[dataCH] * f[dataCH].attrs['VResolution'] + f[dataCH].attrs['VOffset'] + dataMean
                hResolution = f[dataCH].attrs['HResolution']
                hOffset = f[dataCH].attrs['HOffset']
            with h5py.File( fnOut ) as f:                
                if dataCH in f:
                    del f[dataCH]
                f[dataCH] = dataSimulatedNoise
                f[dataCH].attrs['HResolution'] = hResolution
                f[dataCH].attrs['HOffset'] = hOffset
                f[dataCH].attrs['VResolution'] = 1
                f[dataCH].attrs['VOffset'] = 0
            noiseSimulationFilePathNames.append( fnOut )
            del dataSimulatedNoise  
    return noiseSimulationFilePathNames
    
    
def voltage_to_dl( fn, lorentzFwhm, lorentzMaxV, clipAtMinV, cavLength, cavWavelength, 
                  detrend=None, detrendSmoothWindowTime = 5.0, detrendDownsampleN = 50,
                  doPlot=True, dataCH = 'CH0' ):
    with h5py.File( fn ) as f:    
        data = f[dataCH][:,0] * f[dataCH].attrs['VResolution'] + f[dataCH].attrs['VOffset']    
        timeResolution = f[dataCH].attrs['HResolution']    
    
    s = { 'fileName': os.path.basename( fn ), 
         'timeResolution': timeResolution,
         'len': len( data ),
         'min': np.amin( data ),
         'max': np.amax( data ),
         'mean': np.mean( data ),
         'lorentzFwhm': lorentzFwhm,
         'lorentzMaxV': lorentzMaxV,
         'clipAtMinV': clipAtMinV,
         'cavLength': cavLength,
         'cavWavelength': cavWavelength,
         'dlDetrend': detrend ,
         'dlDetrendSmoothWindowTime': detrendSmoothWindowTime, 
         'dlDetrendDownsampleN': detrendDownsampleN }
        
    cavFreq = scipy.constants.c / cavWavelength
    
    if s['min'] < clipAtMinV:
        print 'Data minimum {:.3f} V below specified clipping {:.3f}, clipping data below this voltage.'.format(
        np.amin( data ), clipAtMinV)
        data[ data < clipAtMinV] = clipAtMinV
    
    if s['max'] > lorentzMaxV:
        print 'Data maximum {:.3f} V above specified lorentz maximum {:.3f}, adjusting voltage.'.format(
        s['max'], lorentzMaxV)
        lorentzMaxV = s['max'] + 1e-9    
            
    dataRMS = rms_deviation( data )    
    dl = lorentz_voltage_to_detuning( data, s['timeResolution'], lorentzFwhm, lorentzMaxV, doPlot=doPlot ) * cavLength / cavFreq             
    del data
       
    if detrend == 'linear':
        # 'I should take a look at this code' # 
        # TODO: Find memory-efficient detrend
        N = 10
        t = s['timeResolution'] * np.arange( len( dl ) )        
        slope, intercept = scipy.stats.linregress( t[::N], dl[::N])[:2]
        dl = dl - ( slope * t + intercept )                        
    elif  detrend == 'conv':
        dl = detrend_conv( dl, smoothWindowTime = detrendSmoothWindowTime, downsampleN = detrendDownsampleN )
    elif detrend != None:                        
        dl = scipy.signal.detrend( dl, type=detrend )        
    
    dlRMS = rms_deviation( dl )
    
    if doPlot:
        print '{} min {:5.3f} V, max {:5.3f} V, mean {:5.3f} V, (lw {:.1f} GHz) rms vibrations {:.3f} V, {:.3f} nm'.format( 
        s['fileName'], s['min'], s['max'], s['mean'], lorentzFwhm*1e-9, dataRMS, dlRMS*1e9 )                    
    
    s['dVRMS'] = dataRMS
    s['dlRMS'] = dlRMS
    with h5py.File( fn + '_analysis.h5' ) as f:    
        if 'dl' in f:
            del f['dl']
        f['dl'] = dl        
        for key, value in s.iteritems():
            f.attrs[key] = value        
    return dl    

def compute_spectrum( fn ):
    s = {}
    with h5py.File( fn + '_analysis.h5' ) as f:    
        for key, value in f.attrs.iteritems():
            s[key] = value
        dl = f['dl'][:]    
    freqPow, power = scipy.signal.periodogram( dl, fs = 1/s['timeResolution'] )
    s['frequencyPowerDF'] = freqPow[1] - freqPow[0]    
    with h5py.File( fn + '_analysis.h5' ) as f:    
        if 'power' in f:
            del f['power']
        f['power'] = power        
        for key, value in s.iteritems():
            f.attrs[key] = value    
    return power, freqPow

def plot_dl_periodograms( fns, maxFreq, doNormalize=False ):
    if isinstance(fns, basestring):
        fns = [ fns ]        
    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(1,1,1)
    # ax.set_color_cycle([u'#1f77b4', u'#1f77b4', u'#ff7f0e', u'#ff7f0e', u'#2ca02c', u'#2ca02c', u'#d62728', u'#d62728', u'#9467bd', u'#9467bd', u'#8c564b',u'#8c564b', u'#e377c2', u'#e377c2',  u'#7f7f7f',  u'#7f7f7f', u'#bcbd22', u'#bcbd22', u'#17becf', u'#17becf'])    
    for fn in fns:   
        if not dataset_in_h5file( fn + '_analysis.h5', 'power' ):                
            raise NameError('power dateset does not exist in file' + fn + '_analysis.h5' ) 
        s = {}
        with h5py.File( fn + '_analysis.h5' ) as f:    
            for key, value in f.attrs.iteritems():
                s[key] = value            
            power = f['power'][:]                  
        freqPow = s['frequencyPowerDF'] * np.arange( len( power) )       
        cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=maxFreq  )
        if doNormalize:
            plt.plot( cumFreqs*1e-3, cumPow / (fullPow - dcPow), label=fn )                    
        else:
            plt.plot( cumFreqs*1e-3, 3 * np.sqrt( cumPow ) * 1e9, label=fn )        
        print '{}, full noise power {:.3f} nm/sqrt(Hz), dc frequency {} Hz, relative DC power {:.3f}'.format( s['fileName'], np.sqrt( fullPow ) *1e9, 0.1, dcPow / fullPow )     
    ax.grid()
    ax_add_minortick( ax )
    plt.xlim( -0.1, cumFreqs[-1] *1e-3 )
    plt.xlabel('freq (kHz)')
    if doNormalize:
        plt.ylabel('Normalized integrated noise power')
    else:
        plt.ylabel('3 * sqrt( Integrated noise power ) (nm)')        
    plt.legend(loc=4)    
    plt.show()
    plt.close() 

    
def plot_lowpass_drifts( fns, cutoff=1.0, order=3, downsample=100):
    # Note, butter filter becomes unstable when cutoff is too low or order is too high.

    def butter_lowpass(cutoff, fs, order=5):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    def butter_lowpass_filtfilt(data, cutoff, fs, order=5):
        b, a = butter_lowpass(cutoff, fs, order=order)    
        y = filtfilt(b, a, data)
        return y

    fig = plt.figure(figsize=(12, 9))        
    
    for fn in fns:    
        if not dataset_in_h5file( fn + '_analysis.h5', 'dl' ):                
            raise NameError('dl dataset does not exist in file ' + fn + '_analysis.h5' ) 
        s = {}
        with h5py.File( fn + '_analysis.h5' ) as f:    
            for key, value in f.attrs.iteritems():
                s[key] = value            
            dl = f['dl'][:]                                
        time = s['timeResolution'] * np.arange(dl.shape[0])         
        dlSmooth = butter_lowpass_filtfilt(data=dl, cutoff=cutoff, fs=1.0/s['timeResolution'],order=order)                
        p = np.polyfit( time[::downsample], dlSmooth[::downsample], 1)    
        plt.plot( time[::downsample], dlSmooth[::downsample]*1e9 - np.mean( dlSmooth[::downsample]*1e9 ), label='{} {:.3f} nm/hr'.format( s['fileName'], p[0]*1e9*3600 ) )    
        plt.plot( time[[0,-1]], np.polyval( p, time[[0,-1]] ), ':' )

    
    plt.grid()
    plt.title( 'low-pass {:.1f} Hz, {:.0f}rd order butter'.format( cutoff, order ) )
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'dl (nm)' )    
    plt.legend()
    plt.show()


    

## Update folder from scope

In [ ]:
# Update data dir
data_dir =  r'D:\measuring\data\20171002\scope\\'
scope_dir = r'O:\20171002'
import os, shutil
files = [file for file in os.listdir(scope_dir) if os.path.isfile(os.path.join(scope_dir, file))]
for file in files:
    if not os.path.exists(os.path.join(data_dir, file)):
        shutil.copy(os.path.join(scope_dir, file), data_dir)

## Measurements September and October 2017

In [ ]:



# Default sttings
noiseFileNames = [] 
otherFileNames = []
cavWavelength = 637.0e-9
clipAtMinV = 0.002
detrend = 'linear'
timeStart = 0
timeEnd = -1


# Measurment-specific settings (may overwrite defaults)

# HOM in 3.3 um long cavity
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170905/scope'
# vibrationFileNames = [ 'SCOPE_027.wdf.h5', 'SCOPE_029.wdf.h5' ]
# otherFileNames = [ 'SCOPE_022.wdf.h5', 'SCOPE_023.wdf.h5', 'SCOPE_024.wdf.h5', 'SCOPE_025.wdf.h5', 
#                    'SCOPE_026.wdf.h5', 'SCOPE_028.wdf.h5', 
#                    'SCOPE_030.wdf.h5', 'SCOPE_031.wdf.h5', 'SCOPE_032.wdf.h5', 'SCOPE_033.wdf.h5',  'SCOPE_034.wdf.h5' ]
# lorentzFwhm = 60.0e9      # wild guess, higher-order-mode
# lorentzMaxV = 1.104       # max from data
# cavLength = 3.3e-6    


# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170914/scope'
# vibrationFileNames = [ 'SCOPE_017.wdf.h5', 'SCOPE_019.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_013.wdf.h5' ]
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements 14 sept
# lorentzMaxV = 0.1515    
# cavLength = 19.3e-6    

# Flank position & Electronic filters
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope'
# vibrationFileNames = [ 'SCOPE_007.wdf.h5', 'SCOPE_008.wdf.h5', 'SCOPE_009.wdf.h5',  'SCOPE_011.wdf.h5', 'SCOPE_012.wdf.h5', 'SCOPE_020.wdf.h5' ] # Flank positions
# # vibrationFileNames = [ 'SCOPE_017.wdf.h5', 'SCOPE_018.wdf.h5', 'SCOPE_019.wdf.h5' ] # Electronic filters
# otherFileNames = [ 'SCOPE_006.wdf.h5' ]
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements 14 sept
# lorentzMaxV = 0.086
# cavLength = 19.3e-6    

# JPE Wires
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope'
# vibrationFileNames = [ 'SCOPE_025.wdf.h5', 'SCOPE_026.wdf.h5' ] # JPE wires
# otherFileNames = [ 'SCOPE_024.wdf.h5' ]
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements 14 sept
# lorentzMaxV = 0.176
# cavLength = 19.3e-6  

                         
# Sampling 6.25 MS/s. Test bandwidth filters.
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170921/scope'
# vibrationFileNames = [ 'SCOPE_011.wdf.h5', 'SCOPE_012.wdf.h5', 'SCOPE_013.wdf.h5', 'SCOPE_014.wdf.h5' ] 
# noiseFileNames =     [ 'SCOPE_032.wdf.h5', 'SCOPE_031.wdf.h5', 'SCOPE_033.wdf.h5', 'SCOPE_032.wdf.h5' ]
# otherFileNames =     [ 'SCOPE_026.wdf.h5', 'SCOPE_025.wdf.h5', 'SCOPE_027.wdf.h5' ]   
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements 14 sept
# lorentzMaxV = 0.1945        # Maximum transmission without bandwith filter (SCOPE_025.wdf.h5)    
# cavLength = 19.3e-6  

# # Sampling 6.25 MS/s; low-pass filter 2 MHz. 
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170921/scope'
# vibrationFileNames = [ 'SCOPE_037.wdf.h5', 'SCOPE_038.wdf.h5', 'SCOPE_039.wdf.h5', 'SCOPE_040.wdf.h5' ] 
# noiseFileNames = [ 'SCOPE_033.wdf.h5' ] * len( vibrationFileNames )
# otherFileNames = [ 'SCOPE_027.wdf.h5' ]
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements
# lorentzMaxV = 0.1885        
# cavLength = 19.3e-6    

# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170925/scope'
# vibrationFileNames = [ 'SCOPE_002.wdf.h5', 'SCOPE_006.wdf.h5', 'SCOPE_007.wdf.h5', 'SCOPE_008.wdf.h5', 'SCOPE_009.wdf.h5', 'SCOPE_010.wdf.h5' ] 
# otherFileNames = ['SCOPE_000.wdf.h5' ]
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements
# lorentzMaxV = 0.1965 
# cavLength = 19.3e-6   

# # Many repeated measurements with two Montana settings
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170925/scope'
# vibrationFileNames = [ 'SCOPE_006.wdf.h5', 'SCOPE_007.wdf.h5', 'SCOPE_008.wdf.h5', 'SCOPE_009.wdf.h5', 'SCOPE_010.wdf.h5',   # Normal3
#                        'SCOPE_011.wdf.h5', 'SCOPE_012.wdf.h5', 'SCOPE_013.wdf.h5',                                           # Low, directly
#                        'SCOPE_014.wdf.h5', 'SCOPE_015.wdf.h5', 'SCOPE_016.wdf.h5', 'SCOPE_017.wdf.h5', 'SCOPE_018.wdf.h5', 'SCOPE_019.wdf.h5', 'SCOPE_020.wdf.h5',   # Low, somewhat later
#                        'SCOPE_021.wdf.h5', 'SCOPE_022.wdf.h5', 'SCOPE_023.wdf.h5', 'SCOPE_024.wdf.h5', 'SCOPE_025.wdf.h5' ]  # Low, after stabilization                   
# otherFileNames = ['SCOPE_026.wdf.h5' ]
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements
# lorentzMaxV = 0.1965 
# cavLength = 19.3e-6   


# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170925/scope'
# vibrationFileNames = [ 'SCOPE_021.wdf.h5', 'SCOPE_022.wdf.h5', 'SCOPE_023.wdf.h5', 'SCOPE_024.wdf.h5', 'SCOPE_025.wdf.h5' ] 
# otherFileNames = ['SCOPE_026.wdf.h5', 'SCOPE_027.wdf.h5']
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements
# lorentzMaxV = 0.1925 
# cavLength = 19.3e-6    


# dataDir = r'D:\measuring\data\20170929\scope'
# vibrationFileNames = [ 'SCOPE_004.wdf.csv'] 
# noiseFileNames = [ 'SCOPE_007.wdf.csv' ] 
# lorentzFwhm = 23.0e9        # estimate from slow laserscan measurements
# lorentzMaxV = 0.2755        # 29 sept
# cavLength = 16.4e-6    


# # Spectra measuerments
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171002/scope'
# vibrationFileNames = [ 'SCOPE_004.wdf.h5', 'SCOPE_005.wdf.h5', 'SCOPE_006.wdf.h5', 'SCOPE_009.wdf.h5', 'SCOPE_010.wdf.h5', 
#                        'SCOPE_018.wdf.h5', 'SCOPE_019.wdf.h5', 'SCOPE_020.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_001.wdf.h5' ] 
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 3/10
# lorentzMaxV = 0.2285         
# cavLength = 17.4e-6          

# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171002/scope'
# vibrationFileNames = [ 'SCOPE_012.wdf.h5', 'SCOPE_013.wdf.h5', 'SCOPE_014.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_010.wdf.h5' ] 
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 3/10
# lorentzMaxV = 0.2285         
# cavLength = 17.4e-6          


# # spectra measurements
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171003/scope'
# vibrationFileNames = [ 'SCOPE_001.wdf.h5', 'SCOPE_002.wdf.h5', 'SCOPE_003.wdf.h5', 'SCOPE_004.wdf.h5', 'SCOPE_005.wdf.h5', 'SCOPE_016.wdf.h5' ] 
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 3/10
# lorentzMaxV = 0.2545
# cavLength = 17.4e-6     


# 200s recordings. 
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171003/scope'
# vibrationFileNames = [ 'SCOPE_026.wdf.h5', 'SCOPE_027.wdf.h5', 'SCOPE_028.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_021.wdf.h5', 'SCOPE_025.wdf.h5' ]
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 3/10
# lorentzMaxV = 0.164
# cavLength = 17.4e-6     
# detrend = 'conv'


# # spectra measurements
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope'
# vibrationFileNames = [ 'SCOPE_002.wdf.h5', 'SCOPE_003.wdf.h5', 'SCOPE_004.wdf.h5', 'SCOPE_005.wdf.h5', 'SCOPE_006.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_000.wdf.h5' ]
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 5/10
# lorentzMaxV = 0.206
# cavLength = 18.1e-6     


# 200 s recordings.
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope'
# vibrationFileNames = [ 'SCOPE_007.wdf.h5', 'SCOPE_008.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_000.wdf.h5' ]
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 5/10
# lorentzMaxV = 0.206
# cavLength = 18.1e-6     
# detrend = 'conv'


# # spectra recordings. Montana normal3.
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope'
# vibrationFileNames = [ 'SCOPE_003.wdf.h5', 'SCOPE_004.wdf.h5', 'SCOPE_005.wdf.h5', 'SCOPE_006.wdf.h5', 'SCOPE_007.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_000.wdf.h5' ]
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 5/10
# lorentzMaxV = 0.190
# cavLength = 18.1e-6     


# # spectra recordings. Montana normal3. 2 MHz low-pass.
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope'
# vibrationFileNames = [ 'SCOPE_012.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_015.wdf.h5', 'SCOPE_011.wdf.h5', 'SCOPE_013.wdf.h5', 'SCOPE_014.wdf.h5' ]
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 5/10
# lorentzMaxV = 0.190
# cavLength = 18.1e-6     



# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope'
# vibrationFileNames = [ 'SCOPE_008.wdf.h5', 'SCOPE_010.wdf.h5' ] 
# otherFileNames = [ 'SCOPE_000.wdf.h5' ]
# lorentzFwhm = 20.0e9   # esitmate from slow laserscan measurements on 5/10
# lorentzMaxV = 0.190
# cavLength = 18.1e-6     
# detrend = 'conv'


# # spectra recordings. Montana normal3.
# dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171012/scope'
# vibrationFileNames = [  'SCOPE_000.wdf.h5', 'SCOPE_001.wdf.h5', 'SCOPE_002.wdf.h5', 'SCOPE_003.wdf.h5', 'SCOPE_004.wdf.h5', 'SCOPE_005.wdf.h5' ] 
# otherFileNames = [  'SCOPE_006.wdf.h5' ]
# lorentzFwhm = 22.0e9   # guesitmate from slow laserscan measurements on 3/10 and 5/10, somewhat in-between.
# lorentzMaxV = 0.154
# cavLength = 18.8e-6    

# spectra recordings. Montana toggled normal3 and low
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171012/scope'
vibrationFileNames = [ 'SCOPE_010.wdf.h5', 'SCOPE_011.wdf.h5', 'SCOPE_012.wdf.h5', 
                       'SCOPE_013.wdf.h5', 'SCOPE_014.wdf.h5', 'SCOPE_015.wdf.h5', 'SCOPE_016.wdf.h5', 
                       'SCOPE_017.wdf.h5', 'SCOPE_018.wdf.h5', 'SCOPE_019.wdf.h5' , 
                      'SCOPE_020.wdf.h5','SCOPE_021.wdf.h5'                       ]   
otherFileNames = [  'SCOPE_006.wdf.h5' ]
lorentzFwhm = 22.0e9   # guesitmate from slow laserscan measurements on 3/10 and 5/10, somewhat in-between.
lorentzMaxV = 0.154
cavLength = 18.8e-6    




# Generate filePathNames and convert file.
vibrationFilePathNames = []
noiseFilePathNames = []
otherFilePathNames = []
for fileName in vibrationFileNames:    
    vibrationFilePathNames.append( os.path.join( dataDir, fileName ) )
for fileName in noiseFileNames:    
    noiseFilePathNames.append( os.path.join( dataDir, fileName ) )    
for fileName in otherFileNames:    
    otherFilePathNames.append( os.path.join( dataDir, fileName ) )        

print vibrationFilePathNames
print noiseFilePathNames
print otherFilePathNames

In [ ]:
# Print time-traces and histograms 
# for fn in vibrationFilePathNames + noiseFilePathNames + otherFileNames:    
#     plot_timetrace( fn, 0, 1 )

plot_histograms( vibrationFilePathNames + noiseFilePathNames + otherFilePathNames )



In [ ]:
# Estimate noise data
noiseSimulationFilePathNames = simulate_noise_data( vibrationFilePathNames, noiseFilePathNames ) 
    

In [ ]:
# Compute dl and RMS values
skipIfExists = False
for fn in vibrationFilePathNames + noiseSimulationFilePathNames:    
    if skipIfExists and dataset_in_h5file( fn + '_analysis.h5', 'dl' ) :
        continue            
    voltage_to_dl( fn, lorentzFwhm, lorentzMaxV, clipAtMinV, cavLength, cavWavelength, detrend=detrend )
    

In [ ]:
# Compute spectra 
skipIfExists = False
for fn in vibrationFilePathNames + noiseSimulationFilePathNames:   
    if skipIfExists and dataset_in_h5file( fn + '_analysis.h5', 'power' ) :
        continue           
    compute_spectrum( fn )

In [ ]:
# Print summaries
for fn in vibrationFilePathNames + noiseSimulationFilePathNames:  
    s = read_h5file_attrs_as_dict( fn + '_analysis.h5' )            
    print '{} min {:5.3f} V, max {:5.3f} V, mean {:5.3f} V, (lw {:.1f} GHz) rms vibrations {:.3f} V, {:.3f} nm'.format( 
        s['fileName'], s['min'], s['max'], s['mean'], s['lorentzFwhm']*1e-9, s['dVRMS'], s['dlRMS']*1e9 )                    

# Plot cummulative PSD
maxFreq = 16e3 # 0.5e3
plot_dl_periodograms( vibrationFilePathNames + noiseSimulationFilePathNames, maxFreq )

    

In [ ]:
print matplotlib.rcParams['axes.color_cycle']


[u'#1f77b4', u'#1f77b4', u'#ff7f0e', u'#ff7f0e', u'#2ca02c', u'#2ca02c', u'#d62728', u'#d62728', u'#9467bd', u'#9467bd', u'#8c564b',u'#8c564b', u'#e377c2', u'#e377c2',  u'#7f7f7f',  u'#7f7f7f', u'#bcbd22', u'#bcbd22', u'#17becf', u'#17becf']

## Plot PSDs of different files

In [ ]:


# # Test repeatibility. Measurements with same cavity at various dates. Sept 14 - 25
# plotVibrationFilePathNames = [ 
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170914/scope/SCOPE_017.wdf.h5', 
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope/SCOPE_011.wdf.h5',
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope/SCOPE_018.wdf.h5',
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope/SCOPE_025.wdf.h5',        
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170921/scope/SCOPE_037.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170925/scope/SCOPE_007.wdf.h5',  # 19.4 um cavity, montana normal    
#     ]


# # Test repeatibility. Measurements with same cavity at various dates. Oct 2-10. 
# plotVibrationFilePathNames = [ 
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171002/scope/SCOPE_010.wdf.h5',  # Montana Normal3    
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171002/scope/SCOPE_019.wdf.h5',  # Montana Low    
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171003/scope/SCOPE_002.wdf.h5',  # Montana Low
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_005.wdf.h5',  # Montana Low
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_007.wdf.h5',  # Montana Normal3    
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171012/scope/SCOPE_004.wdf.h5'   # Montana Normal3    
#     ]


# # Test montana compressor Oct 5 and Oct 10
# plotVibrationFilePathNames = [ 
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_002.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_003.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_004.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_005.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_006.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_003.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_004.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_005.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_006.wdf.h5',  
#     '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_007.wdf.h5'
#     ]


# Compare cavity types (glued fiber, HOM, normal). All montana in normal3 setting. 
plotVibrationFilePathNames = [ 
    '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170905/scope/SCOPE_029.wdf.h5',  # 3.3 um cavity, HOM, cool-down #1
    '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170921/scope/SCOPE_037.wdf.h5',  # ~ 20 um cavity, cool-down #1    
    '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171002/scope/SCOPE_009.wdf.h5',  # 17.4 um cavity, fiber glued, cool-down #2
    '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_007.wdf.h5',  # 18.1 um cavity, fiber glued, cool-down #3
    '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171012/scope/SCOPE_004.wdf.h5'   # 18.8 um cavity, fiber glued, cool-down #4
    ]



# Print summaries
for fn in plotVibrationFilePathNames:  
    s = read_h5file_attrs_as_dict( fn + '_analysis.h5' )            
    print '{} min {:5.3f} V, max {:5.3f} V, mean {:5.3f} V, (lw {:.1f} GHz) rms vibrations {:.3f} V, {:.3f} nm'.format( 
        s['fileName'], s['min'], s['max'], s['mean'], s['lorentzFwhm']*1e-9, s['dVRMS'], s['dlRMS']*1e9 )                    

# Plot cummulative PSD
maxFreq = 0.250e3    
plot_dl_periodograms( plotVibrationFilePathNames, maxFreq, doNormalize=False )



## Analyze drift (files 18 September, filter not ok for other files )

In [ ]:
# analyze drift
# Used this script for measurements 18 Sept, but it fails for other measurements.

plot_lowpass_drifts( vibrationFilePathNames, cutoff=100.0, order=3 )

## Analysis to find how much measurement time is avaliable with RMS < 0.1 nm and how to predict these times

In [ ]:
# Functions

def rms_in_bins( x, binN, doCrop=False):
    # I assume dl data is detrended with appropiate detrend.
    if not( doCrop ) and ( len( x ) % binN != 0 ):
        raise ValueError( 'len( x ) should be integer multiple of binN' )
    x = x[:int( np.floor( len( x ) / binN ) * binN )]
    return np.sqrt( (x.reshape(-1, binN) ** 2 ).mean(axis=1) )





In [ ]:
# Load data

# Files with Montana low setting
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171003/scope/SCOPE_026.wdf.h5'
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171003/scope/SCOPE_027.wdf.h5'
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171003/scope/SCOPE_028.wdf.h5'
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_007.wdf.h5'
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_008.wdf.h5'

# Files with Montana normal setting
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171002/scope/SCOPE_012.wdf.h5'
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171002/scope/SCOPE_013.wdf.h5'
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171002/scope/SCOPE_014.wdf.h5'
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_008.wdf.h5'
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_010.wdf.h5'

# Other tests
fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170925/scope/SCOPE_007.wdf.h5'


fileTitle = fn[-31:]

s = {}
with h5py.File( fn + '_analysis.h5' ) as f:    
    for key, value in f.attrs.iteritems():
        s[key] = value
    dl = f['dl'][:]    
timeResolution = s['timeResolution'][0]


In [ ]:
# Divide data in 50 ms bins (regardless Montana cycle period) and compute fraction of bins with RMS below 0.1 nm.

binTime = 0.050
dlBinnedRMS = rms_in_bins( dl, int( binTime / s['timeResolution'] ) )
t = np.arange( len( dlBinnedRMS ) ) * binTime

plt.figure(figsize=(12, 9))    
plt.plot( t, dlBinnedRMS * 1.0e9 )
plt.xlabel( 'time (s)' )
plt.ylabel( 'RMS value of {:.3f} s bins (nm)'.format( binTime ) )
plt.title( '{}\nNumber of bins with RMS below 0.1 nm: {:.1f} %'.format( fileTitle, 100.0 * ( dlBinnedRMS < 0.1e-9 ).sum() / len( dlBinnedRMS )  ) )
plt.grid()
plt.show()
plt.close()


In [ ]:
# Settings
# syncPeriodTime = 1.249661060 # Montana low setting
syncPeriodTime = 0.999721120 # Montana normal setting
binTime = 0.050 

In [ ]:
# Plot RMS values, taking RMS over 50 ms bins and then RMS over same position within Montana cycle

syncPeriodN = int( syncPeriodTime / timeResolution ) 
binN = int( binTime / timeResolution )

numberOfSyncPeriods = int( np.floor( len( dl ) / syncPeriodN ) )
numberOfBinsPerPeriod = int( np.floor( syncPeriodN / binN ) )
dlPeriodBinRMS = np.zeros( (numberOfBinsPerPeriod, numberOfSyncPeriods) )
for i in range( numberOfSyncPeriods ):
    dlPeriodBinRMS[:,i] = rms_in_bins( dl[i:i+syncPeriodN], binN, doCrop=True )
dlPeriodsRMS = np.sqrt( (dlPeriodBinRMS ** 2 ).mean(axis=1) )

plt.figure(figsize=(12, 9))  
plt.plot( np.arange( len( dlPeriodBinRMS ) ) * binTime, dlPeriodsRMS*1e9 )
plt.xlabel( 'time in Montana cycle (s)' )
plt.ylabel( 'RMS over all cycles in {:.3f} s bin'.format( binTime ) )
plt.title( fileTitle )
plt.grid()
plt.show()
plt.close()

# Ok this is not what I expected, I expected two kicks
# Middle kick occurs sometimes. How often?


In [ ]:
# Find most noisy and most quiet Montana cylces. 
# Plot RMS value per Montana cycle period
# Plot 50 ms binned RMS values for most noisy and most quiet Montana cycle periods.

syncPeriodN =  int( syncPeriodTime / timeResolution )
dlPeriodsRMS = rms_in_bins( dl, syncPeriodN, doCrop=True )
t = np.arange( len( dlBinnedRMS ) ) * syncPeriodTime

plt.figure(figsize=(12, 9))    
plt.plot( np.arange( len( dlPeriodsRMS ) ) * syncPeriodTime, dlPeriodsRMS * 1.0e9 )
plt.xlabel( 'time (s)' )
plt.ylabel( 'RMS value of {:.3f} s bins (nm)'.format( syncPeriodTime ) )
plt.title( '{}\nNumber of bins with RMS below 0.1 nm: {:.1f} %'.format( fileTitle, 100.0 * ( dlPeriodsRMS < 0.1e-9 ).sum() / len( dlPeriodsRMS )  ) )
plt.grid()
plt.show()
plt.close()

sortInd = np.argsort( dlPeriodsRMS )

plotNumberOfPeriods = 10# 25

plt.figure(figsize=(12, 9))   
for i in range( len( dlPeriodsRMS ) - plotNumberOfPeriods, len( dlPeriodsRMS )):    
    dlPeriodBinnedRMS = rms_in_bins( dl[ sortInd[i] * syncPeriodN : ( sortInd[i] + 1 ) * syncPeriodN ], int( binTime / timeResolution ), doCrop=True )    
    plt.plot( np.arange( len( dlPeriodBinnedRMS ) ) * binTime, dlPeriodBinnedRMS * 1.0e9, label='#{:03.0f} {:1f} s'.format( i, sortInd[i] * syncPeriodN * timeResolution ) )
plt.xlabel( 'time (s)' )
plt.ylabel( 'RMS value of {:.3f} s bins (nm)'.format( binTime ) )
plt.title( '{} Most noisy periods'.format( fileTitle ) )
# plt.legend()
plt.grid()
plt.show()
plt.close()    

plt.figure(figsize=(12, 9))   
for i in range( plotNumberOfPeriods ):
    dlPeriodBinnedRMS = rms_in_bins( dl[ sortInd[i] * syncPeriodN : ( sortInd[i] + 1 ) * syncPeriodN ], int( binTime / timeResolution ), doCrop=True )    
    plt.plot( np.arange( len( dlPeriodBinnedRMS ) ) * binTime, dlPeriodBinnedRMS * 1.0e9, label='#{:03.0f} {:1f} s'.format( i, sortInd[i] * syncPeriodN * timeResolution ) )
plt.xlabel( 'time (s)' )
plt.ylabel( 'RMS value of {:.3f} s bins (nm)'.format( binTime ) )
plt.title( '{} Most quiet periods'.format( fileTitle ) )
# plt.legend()
plt.grid()
plt.show()
plt.close()    


for i in [ 0, 1, 2, len( dlPeriodsRMS ) - 3, len( dlPeriodsRMS ) - 2, len( dlPeriodsRMS ) - 1 ]:    
    dlPeriodBinnedRMS = rms_in_bins( dl[ sortInd[i] * syncPeriodN : ( sortInd[i] + 1 ) * syncPeriodN ], int( binTime / timeResolution ), doCrop=True )    
    plt.plot( np.arange( len( dlPeriodBinnedRMS ) ) * binTime, dlPeriodBinnedRMS * 1.0e9, label='#{:03.0f} {:1f} s'.format( i, sortInd[i] * syncPeriodN * timeResolution ) )
plt.xlabel( 'time (s)' )
plt.ylabel( '{}\nRMS value of {:.3f} s bins (nm)'.format( fileTitle, binTime ) )
plt.title( fileTitle + '\n Most noisy and most quiet periods' )
plt.legend()
plt.grid()
plt.show()
plt.close()    


# The plots below 



In [ ]:
# Manually choose two quiet times (A and B). Compute RMS values in these periods and also of entire signal.

# For Montana low setting
# quietTimeA1 = 0.25
# quietTimeA2 = 0.65
# quietTimeB1 = 0.95
# quietTimeB2 = 0.02 + syncPeriodTime

# For Montana normal setting
quietTimeA1 = 0.25
quietTimeA2 = 0.53
quietTimeB1 = 0.8
quietTimeB2 = 0.02 + syncPeriodTime


plotNumberOfPeriods = len( sortInd ) # 5
plt.figure(figsize=(12, 9))   
plt.plot( (quietTimeA1 % syncPeriodTime ) * np.ones(2), [0, 0.25], 'k:' ) 
plt.plot( (quietTimeA2 % syncPeriodTime ) * np.ones(2), [0, 0.25], 'k:' ) 
plt.plot( (quietTimeB1 % syncPeriodTime ) * np.ones(2), [0, 0.25], 'k:' ) 
plt.plot( (quietTimeB2 % syncPeriodTime ) * np.ones(2), [0, 0.25], 'k:' )    
for i in range( plotNumberOfPeriods ):
    dlPeriodBinnedRMS = rms_in_bins( dl[ sortInd[i] * syncPeriodN : ( sortInd[i] + 1 ) * syncPeriodN ], int( binTime / timeResolution ), doCrop=True )    
    plt.plot( np.arange( len( dlPeriodBinnedRMS ) ) * binTime, dlPeriodBinnedRMS * 1.0e9, label='{:1f} s'.format( sortInd[i] * syncPeriodN * timeResolution ) )
plt.xlabel( 'time (s)' )
plt.ylabel( 'RMS value of {:.3f} s bins (nm)'.format( binTime ) )
plt.title( fileTitle + ' all periods' )
plt.grid()
plt.show()
plt.close()    


quietTimeA1Ind = int( quietTimeA1 / timeResolution )
quietTimeA2Ind = int( quietTimeA2 / timeResolution )
quietTimeB1Ind = int( quietTimeB1 / timeResolution )
quietTimeB2Ind = int( quietTimeB2 / timeResolution )

numberOfPeriods = int( np.floor( len( dl ) / syncPeriodN ) ) - 1 

quietTimeARMSs = np.zeros( numberOfPeriods )
quietTimeBRMSs = np.zeros( numberOfPeriods )

for i in range( numberOfPeriods ):        
    quietTimeARMSs[i] = np.sqrt( (dl[ i * syncPeriodN + quietTimeA1Ind : i * syncPeriodN + quietTimeA2Ind ] ** 2 ).mean() )
    quietTimeBRMSs[i] = np.sqrt( (dl[ i * syncPeriodN + quietTimeB1Ind : i * syncPeriodN + quietTimeB2Ind ] ** 2 ).mean() )
    
allTimeRMS = np.sqrt( ( dl[ quietTimeA1 : numberOfPeriods * syncPeriodN + quietTimeA1 ] ** 2 ).mean() )
quietTimeARMS = np.sqrt( (quietTimeARMSs ** 2 ).mean() )
quietTimeBRMS = np.sqrt( (quietTimeBRMSs ** 2 ).mean() )


                      
print 'Quiet time A fraction {:.1f} %, quiet time B fraction {:.1f} %'.format( 
    100.0 * ( quietTimeA2 - quietTimeA1 ) / syncPeriodTime, 
    100.0 * ( quietTimeB2 - quietTimeB1 ) / syncPeriodTime )


print '{} RMS all: {:.3f} nm, A: {:.3f} nm, B: {:.3f} nm. Fraction RMS < 0.1 nm, A: {:.1f} %, B: {:.1f} %'.format( 
    fileTitle, 
    allTimeRMS * 1e9, 
    quietTimeARMS * 1e9, 
    quietTimeBRMS * 1e9,
    100.0 * ( quietTimeARMSs < 0.1e-9 ).sum() / numberOfPeriods,
    100.0 * ( quietTimeBRMSs < 0.1e-9 ).sum() / numberOfPeriods ) 
    





In [ ]:
# Export microphone recording of most silent and most noisy Monana cycles as wav file.

from scipy.signal import butter, lfilter
from scipy.signal import freqz
from IPython.display import Audio
from scipy.io import wavfile


def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def downsample_mean( data, downSampleFactor):
    return data[ : int( np.floor( len( data ) / downSampleFactor ) * downSampleFactor ) ].reshape(-1, downSampleFactor).mean(axis=1)

downSampleFactor = 5
lowcut = 20.0
highcut = 28.0e3

sMic = {}
micCh = 'CH2'
with h5py.File( fn ) as f:    
    for key, value in f[micCh].attrs.iteritems():
        sMic[key] = value            
    dataMic = f[micCh][:,0] * f[micCh].attrs['VResolution'] + f[micCh].attrs['VOffset']    
    timeResolution = f[micCh].attrs['HResolution'][0] 

plotNumberOfPeriods = 10
for i in range( plotNumberOfPeriods ) + range( len( sortInd ) - plotNumberOfPeriods, len( sortInd )):    
    # periodInd = range( max( sortInd[i] - 1, 0 ) * syncPeriodN, min( sortInd[i] + 2, len( dl ) / syncPeriodN ) * syncPeriodN )    
    periodInd = range( sortInd[i] * syncPeriodN, ( sortInd[i] + 1 ) * syncPeriodN )    
    micDataFilteredDownsampled = downsample_mean( butter_bandpass_filter(dataMic[periodInd], lowcut, highcut, 1.0 / timeResolution, order=3), downSampleFactor )                
    wavfile.write( fn + '_mic_nr{:03.0f}_time{:.0f}ms.wav'.format( i, sortInd[i] * syncPeriodTime * 1e3 ), 
                   1.0/(downSampleFactor * timeResolution), 
                   np.int16(micDataFilteredDownsampled/np.max(np.abs(micDataFilteredDownsampled)) * 32767) )    


    


In [ ]:
Audio( micDataFilteredDownsampled, rate=1.0/(downSampleFactor * timeResolution) )



In [ ]:
1.0/(downSampleFactor * timeResolution)

## Analyze periodicity of montana low setting

In [ ]:
# fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171005/scope/SCOPE_008.wdf.h5'
fn = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20171010/scope/SCOPE_009.wdf.h5'

s = {}
with h5py.File( fn ) as f:    
    for key, value in f['CH1'].attrs.iteritems():
        s[key] = value
    dataSync = f['CH1'][:,0]        

triggerV = (dataSync.max() - dataSync.min()) / 2.0
triggerInd = np.where( np.logical_and(dataSync[:-1] < triggerV, dataSync[1:] > triggerV ) )[0]
triggerDInd = triggerInd[1:] - triggerInd[:-1]
triggerdIndMean = triggerDInd.mean()
triggerDT = triggerDInd * s['HResolution']
triggerDTMean = triggerDT.mean()
outOfSyncT = triggerDTMean % ( (triggerInd[-1] - triggerInd[0]) * s['HResolution'][0] )
if outOfSyncT > 0.5 * triggerDTMean:
    outOfSyncT = triggerDTMean - outOfSyncT
print 'Montana sync signal period mean {:.9f} s, time resolution {:.1f} us, std {:.1f} us, max-min {:.1f} us, out-of-sync of last trigger {:.1f} us'.format( triggerDTMean, s['HResolution'][0]*1e6, triggerDT.std() *1e6, (triggerDT.max()- triggerDT.min())*1e6, outOfSyncT*1e6 )

print 'Montana sync signal period mean {:.0f} samples with time resolution {:.1f} us'.format( triggerdIndMean, s['HResolution'][0]*1e6 )

plt.hist( triggerDT )
plt.xlabel( 'trigger time period (s)' )
plt.ylabel( 'number of occurances (s)' )


In [ ]:
print 1 - 0.999720608

## I cleaned up the code untill here. Code below needs clean-up and probably remove unused stuff.

In [ ]:
# Influence of photo-diode noise
# Plot relative noise around 50 Hz as function of mean voltage in time-trace.
# Shows clear correlation.

dataMeans = np.zeros( len( vibrationFileNames) )
dlRelative50Hz = np.zeros( len( vibrationFileNames) )
dlRMSs = np.zeros( len( vibrationFileNames) )

for i in range( len( vibrationFileNames ) ):  
    fileName = vibrationFileNames[i]
    data = dataDict[fileName]
    dataMeans[i] = np.mean( data[:,0] )
    dl = dlDict[fileName] 
    dlRMSs[i] = rms_deviation( scipy.signal.detrend( dl ) )
    power = powerDict[fileName]
    freqPow = freqPowDict[fileName]    
    i1 = np.argmin( np.abs( freqPow - 45.0 ) )
    i2 = np.argmin( np.abs( freqPow - 55.0 ) )
    fullPow = scipy.integrate.trapz(power,freqPow,axis=0)
    pow50Hz = scipy.integrate.trapz(power[i1:i2],freqPow[i1:i2],axis=0)           
    dlRelative50Hz[i] = pow50Hz / fullPow
    

plt.plot( dataMeans, dlRelative50Hz, '.' )
plt.xlabel( 'mean voltage V ')
plt.ylabel( 'noise around 50 Hz, relative' )
plt.grid()
plt.show()    
    
    
plt.plot( dataMeans, dlRMSs*1e9, '.' )
plt.xlabel( 'mean voltage (V)')
plt.ylabel( 'rms vibrations (nm)' )
plt.grid()
plt.show()        

In [ ]:
# RMS values per bin, for measurements 25 Sept with different Montana settings.
# Do not average. 

numberOfBins = 40
binTime = 2.0 / numberOfBins


for fileName in vibrationFileNames:    
    dl = dlDict[fileName]
    dlDetrended = scipy.signal.detrend( dl )
    binnedRMS = np.zeros( numberOfBins )
    for i in range( numberOfBins ):
        binnedRMS[i] = rms_deviation( dlDetrended[ np.logical_and( time > binTime*i, time < binTime*(i+1) ) ] )    
    plt.plot( np.arange(numberOfBins) * binTime + 0.5 * binTime, binnedRMS*1e9, '.-' )
plt.grid()
plt.ylabel( 'rms vibration (nm)' )
plt.xlabel( 'time (s)' )
plt.title( 'rms values for bins of {:.3f} s'.format( binTime ) )
plt.show()


## Measurements 18 September 2017: More detailed analysis of one spectrum

In [ ]:
## More detailed analysis of spectrum
#
# Single time-trace only.
#
# Load data, time, dl, power, freqPow

fileName = 'SCOPE_009.wdf.csv'
data = dataDict[fileName]
timeResolution = timeResolutionDict[fileName]
dl = dlDict[fileName]
power = powerDict[fileName]
freqPow = freqPowDict[fileName] 
time = timeResolution * np.arange(data.shape[0]) 

In [ ]:
## More detailed analysis of spectrum


# Plot histogram
plt.hist( dl*1e9, 100, label='dl')
plt.hist( scipy.signal.detrend( dl )*1e9, 100, label='dl, linear detrend')
plt.legend()
plt.xlabel( 'detuning (nm)' )
plt.grid()
plt.show()

# Detrend dl
dlDetrended = scipy.signal.detrend( dl )
hist, binEdges = np.histogram( dlDetrended, 250)
binCenters = 0.5 * ( binEdges[1:] + binEdges[:-1] )

# Fit gauss and lorentzian curves to histogram
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

def gaus(x,a,x0,sigma):
    return a*exp(-(x-x0)**2/(2*sigma**2))

def fit_gauss(x, y, sigmaGuess):
    popt,pcov = curve_fit(gaus,x,y,p0=[ np.max(y), x[ np.argmax(y) ], sigmaGuess ] )        
    return popt

def lorentzian2(x, p1, p2, p3):
    return lorentzian(x, [p1, p2, p3])

def fit_lorentz(x, y, pGuess):
    popt,pcov = curve_fit(lorentzian2,x,y,p0=pGuess )        
    return popt

sigmaGuess = rms_deviation( dlDetrended )
popt = fit_gauss( binCenters, hist, sigmaGuess )
x = np.linspace( binCenters[0], binCenters[-1], 1000 )
yFit = gaus( x, *popt )

popt2 = fit_lorentz( binCenters, hist, [popt[1], popt[2], popt[0]] )
yFit2 = lorentzian2( x, *popt2 )

plt.plot( binCenters*1e9, hist )
plt.plot( x*1e9, yFit, ':', label='gauss, sigma {:3f} nm'.format( popt[2]*1e9 ) )
plt.plot( x*1e9, yFit2, ':', label='lorentz, fwhm {:3f} nm'.format( popt2[1]*1e9 ) )
plt.xlabel( 'detuning (nm)' )
plt.title( '{}, linear detrend, rms vibrations {:.3f} nm'.format( fileName, sigmaGuess*1e9 ) )
plt.legend()
plt.grid()
plt.show()







In [ ]:

timeStart = 0.105

for w in [5, 1, 0.1, 0.01, 0.001, 0.0001 ]:
    i1 = np.argmax( time > timeStart )
    i2 = np.argmax( time > (timeStart + w) )        
    plt.plot( time[i1:i2] - time[i1], dl[i1:i2]*1e9)    
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'dl (nm)' )
    plt.grid()
    plt.show()
    
timeStart = 0.305
for w in [5, 1, 0.1, 0.01, 0.001, 0.0001 ]:
    i1 = np.argmax( time > timeStart )
    i2 = np.argmax( time > (timeStart + w) )        
    plt.plot( time[i1:i2] - time[i1] + 0.20, dl[i1:i2]*1e9)    
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'dl (nm)' )
    plt.grid()
    plt.show()
    
    

In [ ]:

# Plot time traces
# 1s, at various starting times
for timeStart in [0, 10, 20, 30]:
    i1 = np.argmax( time > timeStart )
    i2 = np.argmax( time > timeStart + 1.0 )
    plt.figure(figsize=(12, 8))
    plt.plot( time[i1:i2], dl[i1:i2]*1e9 )    
    plt.plot( time[i1:i2], dlSmooth1[i1:i2]*1e9, ':', label='low-pass 1.0 Hz, 3rd order butter' )    
    plt.plot( time[i1:i2], dlSmooth2[i1:i2]*1e9, ':', label='low-pass 0.1 Hz, 3rd order butter' )    
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'dl (nm)' )
    plt.title( fileName )
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
numberOfBins = 20
binTime = 1.0 / numberOfBins
binnedRMS = np.zeros( numberOfBins )
timeMod1 = np.mod( time, 1.0 )
for i in range( numberOfBins ):
    binnedRMS[i] = rms_deviation( dlDetrended[ np.logical_and( timeMod1 > binTime*i, timeMod1 < binTime*(i+1) ) ] )
    

plt.plot( np.arange(numberOfBins) * binTime + 0.5 * binTime, binnedRMS*1e9, '.-' )
plt.grid()
plt.ylabel( 'rms vibration (nm)' )
plt.xlabel( 'time (s)' )
plt.title( 'rms values for bins of {:.3f} s'.format( binTime ) )
plt.show()

plt.plot( binnedRMS*1e9, '.-' )
plt.grid()
plt.ylabel( 'rms vibration (nm)' )
plt.xlabel( 'bin number' )
plt.title( 'rms values for bins of {:.3f} s'.format( binTime ) )
plt.show()
    

In [ ]:
plt.figure(figsize=(12, 18))

binTimes = np.arange( 0, int( binTime / timeResolution ) - 1 )

for i in [2, 10, 12, 19]:
    # For each bin
    # Compute periodogram for each of the 10 seconds, then average
    print 'i', i
    for j in np.arange( 9.0 ):
        dlBin = dl[ int( j / timeResolution + i * binTime / timeResolution ) + binTimes ]
        f, p1 = scipy.signal.periodogram( dlBin, fs = 1/timeResolution, detrend = 'linear' )
        if j == 0:
            p = p1
        else:
            p = np.add( p, p1 )
        print 'j', j
    p = p / 10.0
        
    cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( f, p, max_freq=40e3  )
    cumPow2 = cumPow * (fullPow-dcPow) 
    min_ind = np.argmax( f > 0.1 )
    max_ind = np.argmax( f > 40e3 )

    plt.subplot(311)
    plt.plot( f[min_ind:max_ind], 10.0*np.log10( p[min_ind:max_ind] ) )
    plt.xlabel('freq (Hz)')
    plt.ylabel('Noise power (m**2 in dB)')
    plt.grid()
    plt.subplot(312)
    plt.plot( cumFreqs*1e-3, cumPow, label='{:.3f} s'.format( (i + 0.5) * binTime ) )        
    plt.xlabel('freq (kHz)')
    plt.ylabel('Integrated noise power, relative (m**2)')
    plt.legend()
    plt.grid()
    plt.subplot(313)
    plt.plot( cumFreqs*1e-3, cumPow2, label='{:.3f} s'.format( (i + 0.5) * binTime ) )        
    plt.xlabel('freq (kHz)')
    plt.ylabel('Integrated noise power (m**2)')
    plt.legend()
    plt.grid()

plt.show()


In [ ]:
# Plot spectrum 

maxFreq = 32e3


cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=maxFreq  )

min_ind = np.argmax( freqPow > 0.1 )
max_ind = np.argmax( freqPow > maxFreq )

plt.figure(figsize=(12, 12))
plt.subplot(211)
plt.plot( freqPow[min_ind:max_ind]*1e-3, 10.0*np.log10( power[min_ind:max_ind] ) )
plt.xlim( [27, 32] )
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (m**2 in dB)')
plt.grid()
plt.subplot(212)
plt.plot( cumFreqs*1e-3, cumPow )   
plt.xlim( [27, 32] )
plt.ylim( [0.8, 1.0] )
plt.xlabel('freq (kHz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.grid()
plt.show()

print '{}, full noise power {:.3f} nm/sqrt(Hz), dc frequency {} Hz, relative DC power {:.3f}'.format( fileName, np.sqrt( fullPow ) *1e9, 0.1, dcPow / fullPow ) 

In [ ]:


from scipy.fftpack import fft
# Number of sample points
N = len( dlDetrended )
# sample spacing
T = timeResolution
x = np.linspace(0.0, N*T, N)
y = dlDetrended
yf = fft(y)
yfabs = 2.0/N * np.abs(yf[0:N//2])
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)


In [ ]:

min_ind = np.argmax( xf > 16.6e3 )
max_ind = np.argmax( xf > 16.8e3 )


def lorentzian2(x, p1, p2, p3):
    return lorentzian(x, [p1, p2, p3])

def fit_lorentz(x, y, pGuess):
    popt,pcov = curve_fit(lorentzian2,x,y,p0=pGuess )        
    return popt

x = xf[min_ind:max_ind]
y = yfabs[min_ind:max_ind]

popt2 = fit_lorentz( x, y, [x[ np.argmax(y) ], 5, 0.8*np.max(y)] )
yFit2 = lorentzian2( x, *popt2 )

print popt2 

plt.plot( x, y )
# plt.plot( x, yFit )
plt.plot( x, yFit2  )
plt.xlabel('freq (Hz)')
plt.ylabel('Amplitude (fft m)')
plt.grid()
plt.show()

In [ ]:
16700 / 27

In [ ]:
# Influence of photo-diode noise

dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170918/scope'
if ( 'fileName' in locals() ) or ( fileName != 'SCOPE_022.wdf.csv' ):
    fileName = 'SCOPE_022.wdf.csv'
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )   

i1 = 0
i2 = np.argmax( time > 1 )
plt.figure(figsize=(16, 8))
plt.subplot(121)
plt.plot( time[i1:i2], data[i1:i2,0] )
plt.grid()
plt.xlabel( 'time (s)' )
plt.ylabel( 'V' )
plt.title( fileName )
plt.subplot(122) 
plt.hist(data[:,0],20,label=fileName)
plt.legend()
plt.grid()
plt.show()
print 'filename {}, min {:10.6f} V, max {:10.6f} V'.format( fileName, np.amin( data[:,0] ), np.amax( data[:,0] ) )  

dataZeroMean = data[:,0] - np.mean( data[:,0] )

plt.figure(figsize=(16, 8))
plt.subplot(121)
plt.plot( time[i1:i2], dataZeroMean[i1:i2] )
plt.grid()
plt.xlabel( 'time (s)' )
plt.ylabel( 'V' )
plt.title( fileName )
plt.subplot(122) 
plt.hist(dataZeroMean,20,label=fileName)
plt.legend()
plt.grid()
plt.show()
print 'filename {} zero-mean, min {:10.6f} V, max {:10.6f} V'.format( fileName, np.amin( dataZeroMean ), np.amax( dataZeroMean ) )  



    
# Cavity settings     
lorentzFwhm = 18.0e9        # theortical estimate from specified mirror reflectivities and clipping losses
lorentzMaxV = 0.0863000001  # maximum in measurements 
cavLength = 19.7e-6
cavWavelength = 637e-9
cavFreq = scipy.constants.c / cavWavelength

# Noise analysis
noiseOffsets = np.linspace( 0.004, 0.080, 15 )
noisedlRMSs = np.zeros( len( noiseOffsets ))


for i in range( len( noiseOffsets ) ):
    # Compute influence of noise at given voltage.
    # Noise is photdiode noise only, with voltage offeset to place it at various positions of lorentz flank.
    d =  dataZeroMean + noiseOffsets[i] 
    df = lorentz_voltage_to_detuning( d, time, lorentzFwhm, lorentzMaxV, doPlot=False )
    dfRMS = rms_deviation( scipy.signal.detrend( df ) )
    dlRMS = dfRMS * cavLength / cavFreq        
    print 'noise offset {:.3f} V, rms vibrations {:.3f} nm'.format( noiseOffsets[i], dlRMS*1e9 )                    
    noisedlRMSs[i] = dlRMS
    
    # spectrum
    dl = df * cavLength / cavFreq
    freqPow, power = scipy.signal.welch( dl, axis = 0, fs = 1/timeResolution, nperseg = len(dl), detrend = 'linear' )      
    cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=500  )
    plt.plot( cumFreqs*1e-3, cumPow, label='offset {:.3f} V'.format( noiseOffsets[i] ) )  
    
plt.grid()
plt.xlabel('freq (kHz)')
plt.ylabel('Integrated noise power, relative (m**2)')
plt.legend()
plt.show()    
    
plt.plot( noiseOffsets, noisedlRMSs*1e9 )
plt.title( 'noise RMS {:.3f} V '.format( rms_deviation( scipy.signal.detrend( d ) ) ) )
plt.xlabel( 'noise mean (V)' )
plt.ylabel( 'noise in length RMS (nm)' )
plt.grid()
plt.show()

    

    

## Measurements 21 September 2017: Different ossiloscope settings

In [ ]:
# Code below is very similar to the analysis of the measurements of Sept 18th, but with slight differences as we
# now have a maximum amplitude, noise, etc per setting.


dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170921/scope'

# sampling 125 kS/s
# fileNamesVibrations = [ 'SCOPE_007.wdf.csv', 'SCOPE_008.wdf.csv', 'SCOPE_009.wdf.csv', 'SCOPE_010.wdf.csv' ] 
# fileNamesBlack      = [ 'SCOPE_035.wdf.csv', 'SCOPE_034.wdf.csv', 'SCOPE_036.wdf.csv', 'SCOPE_035.wdf.csv' ]
# fileNamesMax        = [ 'SCOPE_029.wdf.csv', 'SCOPE_028.wdf.csv', 'SCOPE_030.wdf.csv', 'SCOPE_029.wdf.csv' ]                     
       
# sampling 6.25 MS/s
fileNamesVibrations = [ 'SCOPE_011.wdf.csv', 'SCOPE_012.wdf.csv', 'SCOPE_013.wdf.csv', 'SCOPE_014.wdf.csv' ] 
fileNamesBlack      = [ 'SCOPE_032.wdf.csv', 'SCOPE_031.wdf.csv', 'SCOPE_033.wdf.csv', 'SCOPE_032.wdf.csv' ]
fileNamesMax        = [ 'SCOPE_026.wdf.csv', 'SCOPE_025.wdf.csv', 'SCOPE_027.wdf.csv', 'SCOPE_026.wdf.csv' ]                         
    
fileNames = fileNamesVibrations + fileNamesBlack + fileNamesMax

dataDict = {}
timeResolutionDict = {}

for fileName in fileNames:
    if fileName in dataDict:
        continue    
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )   
    dataDict[fileName] = data
    timeResolutionDict[fileName] = timeResolution

In [ ]:
for fileName in fileNamesVibrations + fileNamesBlack[1:]:    
    data = dataDict[fileName]    
    plt.hist(data[:,0], 20, label=fileName)
plt.legend()
plt.grid()
plt.show()

for fileName in fileNames:    
    data = dataDict[fileName]    
    print '{}, min {:6.3f} V, max {:6.3f} V'.format( fileName, np.amin( data[:,0] ), np.amax( data[:,0] ) )  


In [ ]:

dlDict = {}

for fileName in fileNamesVibrations:    
    data = dataDict[fileName]    
    
    # Analysis    
    lorentzFwhm = 18.0e9        # theortical estimate from specified mirror reflectivities and clipping losses
    lorentzMaxV = 0.193         # maximum over all measurements. 
                                # Note that with different bandwith filter, maximum was 0.181. Not sure which one to use, 
                                # error ~7%
    cavLength = 19.3e-6        
    cavWavelength = 637e-9
    cavFreq = scipy.constants.c / cavWavelength

    # Vibrations
    df = lorentz_voltage_to_detuning( data[:,0], time, lorentzFwhm, lorentzMaxV, doPlot=True )
    dfRMS = rms_deviation( scipy.signal.detrend( df ) )
    dlRMS = dfRMS * cavLength / cavFreq        
    print '{} (lw {:.1f} GHz) rms vibrations {:.3f} V, {:.3f} GHz, {:.3f} nm'.format( fileName, lorentzFwhm*1e-9, rms_deviation( data[:,0] ), dfRMS*1e-9 , dlRMS*1e9 )                    
                
    dlDict[fileName] = df * cavLength / cavFreq

In [ ]:
# Add noise data to dlDict --> noise data is estimted from black signal + offset

for i in range( len( fileNamesVibrations ) ):
    # Compute the influence of noise on vibrations. We offset the measured photo-diode noise (black) 
    # with the mean voltage of the actual measurement and with this data compute dlNoise.
    fileNameVibra = fileNamesVibrations[i]
    fileNameBlack = fileNamesBlack[i]
    
    dataVibra = dataDict[fileNameVibra][:,0]    
    dataBlack = dataDict[fileNameBlack][:,0]
    
    dataBlackWithOffset = dataBlack + np.mean( dataVibra )
    
    # Analysis    
    lorentzFwhm = 18.0e9        # theortical estimate from specified mirror reflectivities and clipping losses
    lorentzMaxV = 0.193         # maximum over all measurements. 
                                # Note that with different bandwith filter, maximum was 0.181. Not sure which one to use, 
                                # error ~7%
    cavLength = 19.3e-6        
    cavWavelength = 637e-9
    cavFreq = scipy.constants.c / cavWavelength

    # Vibrations
    df = lorentz_voltage_to_detuning( dataBlackWithOffset, time, lorentzFwhm, lorentzMaxV, doPlot=True )
    dfRMS = rms_deviation( scipy.signal.detrend( df ) )
    dlRMS = dfRMS * cavLength / cavFreq        
    print '{} (lw {:.1f} GHz) rms vibrations {:.3f} V, {:.3f} GHz, {:.3f} nm'.format( fileName, lorentzFwhm*1e-9, rms_deviation( data[:,0] ), dfRMS*1e-9 , dlRMS*1e9 )                    
                
    dlDict[fileNameBlack] = df * cavLength / cavFreq

In [ ]:
print fileNameBlack, np.amax( dataBlack ), dataVibra.shape, dataBlack.shape

In [ ]:
# Compute power

powerDict = {}
freqPowDict = {}  

for fileName in fileNamesVibrations + fileNamesBlack:    
    dl = dlDict[fileName]      
    # freqPow, power = scipy.signal.welch( dl, axis = 0, fs = 1/timeResolution, nperseg = len(dl), detrend = 'linear' ) # too slow. But maybe welch filter with nperseg entire length is weird anyway.
    freqPow, power = scipy.signal.periodogram( dl, fs = 1/timeResolution, detrend = 'linear' )
    powerDict[fileName] = power
    freqPowDict[fileName] = freqPow
                
    

In [ ]:
plt.figure(figsize=(12, 8))

for fileName in fileNamesVibrations + fileNamesBlack:  
    power = powerDict[fileName]  
    freqPow = freqPowDict[fileName]
                
    cumFreqs, cumPow, fullPow, dcPow = calc_cum_pow( freqPow, power, max_freq=3.124e6  )       # 62.5e3
    cumPow2 = cumPow * (fullPow-dcPow)  
    plt.plot( cumFreqs*1e-3, cumPow2, label=fileName )        
    
        
    print '{}, full noise power {:.3f} nm/sqrt(Hz), dc frequency {} Hz, relative DC power {:.3f}'.format( fileName, np.sqrt( fullPow ) *1e9, 0.1, dcPow / fullPow ) 
    
plt.grid()
plt.xlabel('freq (kHz)')
plt.ylabel('Integrated noise power')
plt.legend()
plt.show()